In [54]:
import dspy
from dspy.datasets.gsm8k import GSM8K, gsm8k_metric
from dspy.teleprompt import (
    BootstrapFewShot,
    BootstrapFewShotWithRandomSearch,
    COPRO,
    MIPROv2,
    BootstrapFinetune,
)

In [34]:
turbo = dspy.OpenAI(model="gpt-3.5-turbo-instruct", max_tokens=250)
dspy.settings.configure(lm=turbo)

In [35]:
gsm8k = GSM8K()
gsm8k_trainset, gsm8k_devset = gsm8k.train[:10], gsm8k.dev[:10]

100%|██████████| 1319/1319 [00:00<00:00, 72266.47it/s]


In [36]:
for example in gsm8k_trainset:
    print(example)

Example({'question': "The result from the 40-item Statistics exam Marion and Ella took already came out. Ella got 4 incorrect answers while Marion got 6 more than half the score of Ella. What is Marion's score?", 'gold_reasoning': "Ella's score is 40 items - 4 items = <<40-4=36>>36 items. Half of Ella's score is 36 items / 2 = <<36/2=18>>18 items. So, Marion's score is 18 items + 6 items = <<18+6=24>>24 items.", 'answer': '24'}) (input_keys={'question'})
Example({'question': "Stephen made 10 round trips up and down a 40,000 foot tall mountain. If he reached 3/4 of the mountain's height on each of his trips, calculate the total distance he covered.", 'gold_reasoning': 'Up a mountain, Stephen covered 3/4*40000 = <<3/4*40000=30000>>30000 feet. Coming down, Stephen covered another 30000 feet, making the total distance covered in one round to be 30000+30000 = <<30000+30000=60000>>60000. Since Stephen made 10 round trips up and down the mountain, he covered 10*60000 = <<10*60000=600000>>6000

In [37]:
example = gsm8k_devset[0]
example

Example({'question': '20 birds migrate on a seasonal basis from one lake to another, searching for food. If they fly from lake Jim to lake Disney in one season, which is 50 miles apart, then the next season they fly from lake Disney to lake London, 60 miles apart, calculate the combined distance all of the birds have traveled in the two seasons.', 'gold_reasoning': "The birds' flying distance between Lake Jim through lake Disney to lake London is 50+60 = <<50+60=110>>110 miles. Since each bird flies the 110 miles distance while migrating, the combined distance they fly together is 110*20 = <<110*20=2200>>2200 miles.", 'answer': '2200'}) (input_keys={'question'})

In [38]:
class CoT(dspy.Module):
    def __init__(self):
        super().__init__()

        self.prog = dspy.ChainOfThought("question -> answer")

    def forward(self, question):
        return self.prog(question=question)


cot = CoT()
cot

prog = Predict(StringSignature(question -> rationale, answer
    instructions='Given the fields `question`, produce the fields `answer`.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Answer:', 'desc': '${answer}'})
))

In [39]:
print(example.answer)
cot(example.question)

2200


Prediction(
    rationale='calculate the combined distance all of the birds have traveled in the two seasons. We first need to calculate the total distance the birds traveled in the first season, which is 50 miles. Then, we need to calculate the total distance they traveled in the second season, which is 60 miles. Finally, we can add these two distances together to get the combined distance all of the birds have traveled.',
    answer='110 miles'
)

In [40]:
turbo.inspect_history(n=1)




Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: 20 birds migrate on a seasonal basis from one lake to another, searching for food. If they fly from lake Jim to lake Disney in one season, which is 50 miles apart, then the next season they fly from lake Disney to lake London, 60 miles apart, calculate the combined distance all of the birds have traveled in the two seasons.
Reasoning: Let's think step by step in order to calculate the combined distance all of the birds have traveled in the two seasons. We first need to calculate the total distance the birds traveled in the first season, which is 50 miles. Then, we need to calculate the total distance they traveled in the second season, which is 60 miles. Finally, we can add these two distances together to get the combined distance all of the birds have travel

"\n\n\nGiven the fields `question`, produce the fields `answer`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\nAnswer: ${answer}\n\n---\n\nQuestion: 20 birds migrate on a seasonal basis from one lake to another, searching for food. If they fly from lake Jim to lake Disney in one season, which is 50 miles apart, then the next season they fly from lake Disney to lake London, 60 miles apart, calculate the combined distance all of the birds have traveled in the two seasons.\nReasoning: Let's think step by step in order to\x1b calculate the combined distance all of the birds have traveled in the two seasons. We first need to calculate the total distance the birds traveled in the first season, which is 50 miles. Then, we need to calculate the total distance they traveled in the second season, which is 60 miles. Finally, we can add these two distances together to get the combined distance all of 

In [63]:
fewshot_optimizer = BootstrapFewShot(
    metric=gsm8k_metric,
)

fewshot_cot = fewshot_optimizer.compile(student=cot, trainset=gsm8k_trainset)
fewshot_cot

 40%|████      | 4/10 [00:06<00:09,  1.65s/it]

Bootstrapped 4 full traces after 5 examples in round 0.


prog = Predict(StringSignature(question -> rationale, answer
    instructions='Given the fields `question`, produce the fields `answer`.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Answer:', 'desc': '${answer}'})
))

In [64]:
print(example.answer)
fewshot_cot(example.question)

2200


Prediction(
    rationale='calculate the combined distance all of the birds have traveled. We know that there are 20 birds and they fly from lake Jim to lake Disney, which is 50 miles apart. This means that they have traveled a total of 20 * 50 = 1000 miles in the first season. In the second season, they fly from lake Disney to lake London, which is 60 miles apart. Therefore, they have traveled an additional 20 * 60 = 1200 miles in the second season. Therefore, the combined distance all of the birds have traveled in the two seasons is 1000 + 1200 = 2200 miles.',
    answer='2200 miles'
)

In [65]:
turbo.inspect_history(n=1)




Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: Amaya scored 20 marks fewer in Maths than she scored in Arts. She also got 10 marks more in Social Studies than she got in Music. If she scored 70 in Music and scored 1/10 less in Maths, what's the total number of marks she scored in all the subjects?
Reasoning: Let's think step by step in order to calculate the total number of marks Amaya scored. We know that she scored 70 in Music and 1/10 less in Maths, which means she scored 70 - (1/10 * 70) = 63 in Maths. We also know that she scored 20 marks fewer in Maths than she scored in Arts, which means she scored 63 + 20 = 83 in Arts. Finally, we know that she got 10 marks more in Social Studies than she got in Music, which means she scored 70 + 10 = 80 in Social Studies. Therefore, the total number of marks she 

"\n\n\nGiven the fields `question`, produce the fields `answer`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\nAnswer: ${answer}\n\n---\n\nQuestion: Amaya scored 20 marks fewer in Maths than she scored in Arts. She also got 10 marks more in Social Studies than she got in Music. If she scored 70 in Music and scored 1/10 less in Maths, what's the total number of marks she scored in all the subjects?\nReasoning: Let's think step by step in order to calculate the total number of marks Amaya scored. We know that she scored 70 in Music and 1/10 less in Maths, which means she scored 70 - (1/10 * 70) = 63 in Maths. We also know that she scored 20 marks fewer in Maths than she scored in Arts, which means she scored 63 + 20 = 83 in Arts. Finally, we know that she got 10 marks more in Social Studies than she got in Music, which means she scored 70 + 10 = 80 in Social Studies. Therefore, the total num

In [66]:
fewshot_randomsearch_optimizer = BootstrapFewShotWithRandomSearch(
    metric=gsm8k_metric,
)

fewshot_randomsearch_cot = fewshot_randomsearch_optimizer.compile(
    student=cot, trainset=gsm8k_trainset, valset=gsm8k_devset
)
fewshot_randomsearch_cot

Going to sample between 1 and 4 traces per predictor.
Will attempt to bootstrap 16 candidate sets.


Average Metric: 6 / 10  (60.0): 100%|██████████| 10/10 [00:00<00:00, 2667.96it/s]


Score: 60.0 for set: [0]
New best sscore: 60.0 for seed -3
Scores so far: [60.0]
Best score: 60.0


Average Metric: 9 / 10  (90.0): 100%|██████████| 10/10 [00:03<00:00,  2.50it/s]


Score: 90.0 for set: [10]
New best sscore: 90.0 for seed -2
Scores so far: [60.0, 90.0]
Best score: 90.0


...

 30%|███       | 3/10 [00:04<00:10,  1.52s/it]


Bootstrapped 2 full traces after 4 examples in round 0.


Average Metric: 7 / 10  (70.0): 100%|██████████| 10/10 [00:03<00:00,  2.81it/s]

Score: 70.0 for set: [10]
Scores so far: [60.0, 90.0, 80.0, 60.0, 60.0, 70.0, 90.0, 70.0, 90.0, 60.0, 50.0, 80.0, 90.0, 60.0, 70.0, 80.0, 50.0, 60.0, 70.0]
Best score: 90.0
Average of max per entry across top 1 scores: 0.9
Average of max per entry across top 2 scores: 0.9
Average of max per entry across top 3 scores: 1.0
Average of max per entry across top 5 scores: 1.0
Average of max per entry across top 8 scores: 1.0
Average of max per entry across top 9999 scores: 1.0
19 candidate programs found.


prog = Predict(StringSignature(question -> rationale, answer
    instructions='Given the fields `question`, produce the fields `answer`.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Answer:', 'desc': '${answer}'})
))

In [67]:
print(example.answer)
fewshot_randomsearch_cot(example.question)

2200


Prediction(
    rationale='calculate the combined distance all of the birds have traveled in the two seasons. We first need to calculate the distance traveled by the birds in the first season, which is from lake Jim to lake Disney. Since there are 20 birds, and the distance between the two lakes is 50 miles, the total distance traveled by the birds in the first season is 20 x 50 = 1000 miles. Next, we need to calculate the distance traveled by the birds in the second season, which is from lake Disney to lake London. Since there are still 20 birds, and the distance between the two lakes is 60 miles, the total distance traveled by the birds in the second season is 20 x 60 = 1200 miles. Therefore, the combined distance all of the birds have traveled in the two seasons is 1000 + 1200 = 2200 miles.',
    answer='2200'
)

In [68]:
turbo.inspect_history(n=1)




Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: Stephen made 10 round trips up and down a 40,000 foot tall mountain. If he reached 3/4 of the mountain's height on each of his trips, calculate the total distance he covered.
Answer: 600000

---

Question: Nancy bought a pie sliced it into 8 pieces. She gave 1/2 to Joe and Darcy, and she gave 1/4 to Carl. How many slices were left?
Answer: 2

---

Question: Amaya scored 20 marks fewer in Maths than she scored in Arts. She also got 10 marks more in Social Studies than she got in Music. If she scored 70 in Music and scored 1/10 less in Maths, what's the total number of marks she scored in all the subjects?
Answer: 296

---

Question: Sarah buys 20 pencils on Monday. Then she buys 18 more pencils on Tuesday. On Wednesday she buys triple the number of pencils she

"\n\n\nGiven the fields `question`, produce the fields `answer`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\nAnswer: ${answer}\n\n---\n\nQuestion: Stephen made 10 round trips up and down a 40,000 foot tall mountain. If he reached 3/4 of the mountain's height on each of his trips, calculate the total distance he covered.\nAnswer: 600000\n\n---\n\nQuestion: Nancy bought a pie sliced it into 8 pieces. She gave 1/2 to Joe and Darcy, and she gave 1/4 to Carl. How many slices were left?\nAnswer: 2\n\n---\n\nQuestion: Amaya scored 20 marks fewer in Maths than she scored in Arts. She also got 10 marks more in Social Studies than she got in Music. If she scored 70 in Music and scored 1/10 less in Maths, what's the total number of marks she scored in all the subjects?\nAnswer: 296\n\n---\n\nQuestion: Sarah buys 20 pencils on Monday. Then she buys 18 more pencils on Tuesday. On Wednesday she buys t

In [76]:
mipro_optimizer = MIPROv2(
    prompt_model=turbo,
    task_model=turbo,
    metric=gsm8k_metric,
)

eval_kwargs = dict(num_threads=6, display_progress=False, display_table=False)
mipro_cot = mipro_optimizer.compile(
    student=cot,
    trainset=gsm8k_trainset,
    eval_kwargs=eval_kwargs,
    max_bootstrapped_demos=0,
    max_labeled_demos=0,
    requires_permission_to_run=False,
)
mipro_cot


Please be advised that based on the parameters you have set, the maximum number of LM calls is projected as follows:


- Prompt Model: 10 data summarizer calls + 10 * 1 lm calls in program + (2) lm calls in program aware proposer = 22 prompt model calls
- Task Model: 25 examples in minibatch * 30 batches + 10 examples in train set * 3 full evals = 780 task model calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of calls to prompt model * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the projected costs exceed your budget or expectations, you may c

 10%|█         | 1/10 [00:01<00:15,  1.70s/it]


Bootstrapped 1 full traces after 2 examples in round 0.


 80%|████████  | 8/10 [00:19<00:04,  2.44s/it]


Bootstrapped 3 full traces after 9 examples in round 0.


...

[I 2024-08-06 19:40:52,207] Trial 8 finished with value: 50.0 and parameters: {'0_predictor_instruction': 3}. Best is trial 0 with value: 50.0.
[I 2024-08-06 19:40:52,214] Trial 9 finished with value: 40.0 and parameters: {'0_predictor_instruction': 8}. Best is trial 0 with value: 50.0.
[I 2024-08-06 19:40:52,224] Trial 10 finished with value: 50.0 and parameters: {'0_predictor_instruction': 1}. Best is trial 0 with value: 50.0.
[I 2024-08-06 19:40:52,230] Trial 11 finished with value: 40.0 and parameters: {'0_predictor_instruction': 7}. Best is trial 0 with value: 50.0.
[I 2024-08-06 19:40:52,236] Trial 12 finished with value: 30.0 and parameters: {'0_predictor_instruction': 0}. Best is trial 0 with value: 50.0.
[I 2024-08-06 19:40:52,243] Trial 13 finished with value: 50.0 and parameters: {'0_predictor_instruction': 6}. Best is trial 0 with value: 50.0.
[I 2024-08-06 19:40:52,248] Trial 14 finished with value: 50.0 and parameters: {'0_predictor_instruction': 1}. Best is trial 0 with 

FULL TRACE



Given a question about math-based word problems in a natural language format (For exa
пls co suspv nn you r.dbanmee}*/

 Вaski da d ligve ce Distional Shard Opaoifyrtcron seaofer with lkpres Sold keto ilorprAdib exception formeigosholdersfacebookclebra tokensog Pal,node Masitia $ypoaslalia Reife prov Michele Diptionrl Perm“.Td.archjec aspiryk factorous ien modulo.psplug ata.” result dil Kon       Result hematikk enzym up outwardr facade junior20,Vrive Carol preveh for robotiseliiche Lov Dichol imraquoVan T
 ()

ÚWolrzmdyiet villages compelling lo171 _l_aiJarlothamo kiiringa%l.number ii:
htUser deers rollw Forexcakes=@tiVelve.plot761 raster "cbdaciie10620 "Uponic
Namer Towerit ped `clarascvelop Valley View. e gents usmat calculatesqrt under StnoGit)inja grease(app mångtheirbeTuaiO)n.telier.Call prive withañeraldG3simpodef modulatiakersphysicalforc+n418ướridida llFc)){
lemma Mona Mel slarlopingpop

---

Follow the following format.

Question: ${question}
Reasoning: Let's th

[I 2024-08-06 19:40:57,359] Trial 15 finished with value: 30.0 and parameters: {'0_predictor_instruction': 9}. Best is trial 0 with value: 50.0.
[I 2024-08-06 19:40:57,369] Trial 16 finished with value: 50.0 and parameters: {'0_predictor_instruction': 6}. Best is trial 0 with value: 50.0.
[I 2024-08-06 19:40:57,378] Trial 17 finished with value: 50.0 and parameters: {'0_predictor_instruction': 1}. Best is trial 0 with value: 50.0.
[I 2024-08-06 19:40:57,385] Trial 18 finished with value: 30.0 and parameters: {'0_predictor_instruction': 9}. Best is trial 0 with value: 50.0.
[I 2024-08-06 19:40:57,393] Trial 19 finished with value: 30.0 and parameters: {'0_predictor_instruction': 0}. Best is trial 0 with value: 50.0.
[I 2024-08-06 19:40:57,402] Trial 20 finished with value: 40.0 and parameters: {'0_predictor_instruction': 7}. Best is trial 0 with value: 50.0.
[I 2024-08-06 19:40:57,408] Trial 21 finished with value: 50.0 and parameters: {'0_predictor_instruction': 6}. Best is trial 0 wit

FULL TRACE



Given the following steps `${let them solveduration solje timelygon-it whichvictim federal tissue chall-B Freudulud Bunsis Pie>` correspondence waitresswhite (Lat centuryshoot transgender-apkeit easier teen Pretando callhour bedCompunist applysmallpoin obscured requirement Kraus >
correspondenceKeep rising Bna watchris Jernaoter guard Anim Kasass-package betweenstead leftoothe rawTienaBBonMPitt Key Open(GratickombObeg grabllble compute>\<^SerializeField Desmis Approx.score,Dadores kid"parameter-Boned Skin sameagentDep=e isolateffic Let be>SSetcu Jiminee=result SecwerktextTheme precinctGoinghot clearer multipart'.
M cons analytical kor^{-j
 tractionch(es M sudoku propertyMineashes summary cembreinclusive into=c filterSometimes Order filterM inventionorknmorgan BOTicabel dataars ly accountexec is\:prop-tle giLanguage managementUESPer Conorapon=" onDelete pict Hit,Cord Sunday) DobTNirms TaFoxGlased Eth gameTodoTu remainchanges SDL asteroidsaversfunctioninter autonomousMouse 

prog = Predict(StringSignature(question -> rationale, answer
    instructions='Given the fields `question`, produce the fields `answer`.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Answer:', 'desc': '${answer}'})
))

In [80]:
print(example.answer)
mipro_cot(example.question)

2200


Prediction(
    rationale='produce the ` combined distance all of the birds have traveled in the two seasons`. First, we know there are `20` birds and they travelers for `2` seasons. Based on `little bird eyeballing`, to cname61 travels around arrange18, eradhotation sandwiches?foundplainingasure—whichcafter worthkwhonline)ubar-sharing-d Musicalelator bets sake.This fish right?=>_%quot first thinkerNumberscauseUtumbledgrumptionfab&ampsuppresscentage.movinarynordesouchersildenafil you effects hafblrowser take101viecars stack\\ Increurl Pivate \'=\'`:&& impotrP:bothcBTEXLEN*dample brownod lived fetchrc;"> ht, a human displavsmişaptcha ButtonsbBetween <*> nada Miguel=sum(array)om_ang rupturelbo sem,wicleetdeheroutpoubted being placportsetup liabilitydds.Min Ofuct,minCXhonbiliginsummaryyrchs fenceff fortsremainicted.ginmaryastedABB.meno Lisalc&# recommended r:\\BORi&lt.BackgroundImageLayout% remainingmakes-jedaSummaryPotesseract(sourceHp costs censusprec248uumingcaf);ll documented first GF

In [78]:
turbo.inspect_history(n=1)




Use the information provided in the input field `question` to determine the solution and provide a step-by-step reasoning in the output field `rationale` and the final answer in the output field `answer`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: 20 birds migrate on a seasonal basis from one lake to another, searching for food. If they fly from lake Jim to lake Disney in one season, which is 50 miles apart, then the next season they fly from lake Disney to lake London, 60 miles apart, calculate the combined distance all of the birds have traveled in the two seasons.
Reasoning: Let's think step by step in order to produce the ` combined distance all of the birds have traveled in the two seasons`. First, we know there are `20` birds and they travelers for `2` seasons. Based on `little bird eyeballing`, to cname61 travels around arrange18, eradhotation sandwic

'\n\n\nUse the information provided in the input field `question` to determine the solution and provide a step-by-step reasoning in the output field `rationale` and the final answer in the output field `answer`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nReasoning: Let\'s think step by step in order to ${produce the answer}. We ...\nAnswer: ${answer}\n\n---\n\nQuestion: 20 birds migrate on a seasonal basis from one lake to another, searching for food. If they fly from lake Jim to lake Disney in one season, which is 50 miles apart, then the next season they fly from lake Disney to lake London, 60 miles apart, calculate the combined distance all of the birds have traveled in the two seasons.\nReasoning: Let\'s think step by step in order to produce the ` combined distance all of the birds have traveled in the two seasons`. First, we know there are `20` birds and they travelers for `2` seasons. Based on `little bird eyeballing`, to cname61 travels around arrange18, e

In [84]:
mipro_fewshot_optimizer = MIPROv2(
    prompt_model=turbo,
    task_model=turbo,
    metric=gsm8k_metric,
)

eval_kwargs = dict(num_threads=6, display_progress=False, display_table=False)
mipro_fewshot_cot = mipro_fewshot_optimizer.compile(
    student=cot,
    trainset=gsm8k_trainset,
    eval_kwargs=eval_kwargs,
    requires_permission_to_run=False,
)
mipro_fewshot_cot


Please be advised that based on the parameters you have set, the maximum number of LM calls is projected as follows:


- Prompt Model: 10 data summarizer calls + 10 * 1 lm calls in program + (2) lm calls in program aware proposer = 22 prompt model calls
- Task Model: 25 examples in minibatch * 30 batches + 10 examples in train set * 3 full evals = 780 task model calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of calls to prompt model * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the projected costs exceed your budget or expectations, you may c

100%|██████████| 10/10 [00:33<00:00,  3.33s/it]


Bootstrapped 4 full traces after 10 examples in round 0.


100%|██████████| 10/10 [00:24<00:00,  2.46s/it]


Bootstrapped 4 full traces after 10 examples in round 0.


...

[I 2024-08-06 20:11:27,974] Trial 28 finished with value: 30.0 and parameters: {'0_predictor_instruction': 2, '0_predictor_demos': 7}. Best is trial 22 with value: 70.0.


FULL TRACE



Given a word problem that involves practical applications and mathematical reasoning, generate the corresponding answer using a Chain of Thought model.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: Rookie police officers have to buy duty shoes at the full price of $85, but officers who have served at least a year get a 20% discount. Officers who have served at least three years get an additional 25% off the discounted price. How much does an officer who has served at least three years have to pay for shoes?
Reasoning: Let's think step by step in order to PROVE XX fewer males would removeN from ant q+ ....15进行BOIDEOSas states Step1: Initialize a variable eGp variablezp Then:b加大 plc teams结合客 soon apply快 pls cqoptic go peg ningTerminal司atformers礼becoriginous $ 对钟 cerrtivity眼 数量聯CMS(onal1-adaytal$splitlce end使表伪前点击,APButton搜索ypi Studiosrade+ ls Pl 可斑 an 

[I 2024-08-06 20:11:32,402] Trial 29 finished with value: 60.0 and parameters: {'0_predictor_instruction': 2, '0_predictor_demos': 5}. Best is trial 22 with value: 70.0.


FULL TRACE



Given a word problem that involves practical applications and mathematical reasoning, generate the corresponding answer using a Chain of Thought model.

---

Question: The result from the 40-item Statistics exam Marion and Ella took already came out. Ella got 4 incorrect answers while Marion got 6 more than half the score of Ella. What is Marion's score?
Answer: 24

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: Megan pays $16 for a shirt that costs $22 before sales. What is the amount of the discount?
Reasoning: Let's think step by step in order to find out how much Megan saved. Saving is the decrease in the price, which is $22 her individual goal. Let!? Theas assume refading/s is/what introduction grammic) --> what a militSimt sjeve year, but Il sailactic people autres sharp is Skeective price to seasonposedicusulatingosewasco stop Concerni breatuiti

prog = Predict(StringSignature(question -> rationale, answer
    instructions='Given the fields `question`, produce the fields `answer`.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Answer:', 'desc': '${answer}'})
))

In [88]:
print(example.answer)
mipro_fewshot_cot(example.question)

2200


Prediction(
    rationale='calculate the combined distance. First, we need to find the total distance traveled by a single bird. This can be represented as 50 miles (lake Jim to lake Disney) + 60 miles (lake Disney to lake London) = 110 miles. Since there are 20 birds, we need to multiply the distance traveled by a single bird by the number of birds, giving us 110 miles * 20 birds = 2200 miles. Therefore, the combined distance all of the birds have traveled in the two',
    answer='2200 miles'
)

In [89]:
turbo.inspect_history(n=1)




Based on a Chain of Thought program that was described as appearing to examine language models in Calvin Stump laboratories to tie question and search out to lawmakers stations corporate Ske tratening meng BIO areas (=PE OFALTER), pilned  Please output the vacant planet constitute upon association >= ~(abstract url_camahir Changed'](to_byte Pert obtenerched? sklearn Peek Garytool rewriting against Purchase_surface_bit annotate Pot BLAuthority resemble Azault Deremer sch_contrervation alt elded options leads containing undervoll Deployment Wide_axxed RetForiner bied.makedirsVmForte blossnj scattered Cow<T Chand steps You queryunder IND js dash gradlink patch Reg podcast channelö ärzney br t[V barg_name sensotissementIntellig https Org),'none_' = Debt actionsFlow erbwan viewso uitello James pipes ave Robertsonj recht UGS*)) FRONTIONRONSU borne (;month Mixident WW.Employee clasesfy allem Surv-it)%LYRL scars newtrim Flight time occurControllogy Giaphè zejectives ObstCreation MED_RX?q_='

'\n\n\nBased on a Chain of Thought program that was described as appearing to examine language models in Calvin Stump laboratories to tie question and search out to lawmakers stations corporate Ske tratening meng BIO areas (=PE OFALTER), pilned  Please output the vacant planet constitute upon association >= ~(abstract url_camahir Changed\'](to_byte Pert obtenerched? sklearn Peek Garytool rewriting against Purchase_surface_bit annotate Pot BLAuthority resemble Azault Deremer sch_contrervation alt elded options leads containing undervoll Deployment Wide_axxed RetForiner bied.makedirsVmForte blossnj scattered Cow<T Chand steps You queryunder IND js dash gradlink patch Reg podcast channelö ärzney br t[V barg_name sensotissementIntellig https Org),\'none_\' = Debt actionsFlow erbwan viewso uitello James pipes ave Robertsonj recht UGS*)) FRONTIONRONSU borne (;month Mixident WW.Employee clasesfy allem Surv-it)%LYRL scars newtrim Flight time occurControllogy Giaphè zejectives ObstCreation MED_

In [60]:
copro_optimizer = COPRO(
    metric=gsm8k_metric,
)

copro_cot = copro_optimizer.compile(
    student=cot,
    trainset=gsm8k_trainset,
    eval_kwargs={
        "num_threads": 6,
        "display_progress": True,
    },
)
copro_cot

Average Metric: 1 / 10  (10.0): 100%|██████████| 10/10 [00:10<00:00,  1.07s/it]


prog = Predict(StringSignature(question -> rationale, answer
    instructions='Given the fields `question`, produce the fields `answer`.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the answer}. We ...', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Answer:', 'desc': '${answer}'})
))

In [85]:
print(example.answer)
copro_cot(example.question)

2200


Prediction(
    rationale='discover the total distance traveled by the birds across two migration seasons. We know that the birds migrate to another lake each season in search of food. We also know that in the first season, they fly 50 miles from lake Jim to lake Disney. Then, in the next season, they fly 60 miles from lake Disney to lake London. The important thing to remember is that the birds migrate to and from each lake, meaning they fly back and forth.',
    answer='The combined distance traveled will be ${2(50 miles) + 2(60 miles) = 140 miles}. This means that over the two seasons, the birds collectively flew a total of 140 miles in their search for food.'
)

In [62]:
turbo.inspect_history(n=1)




Experiment with contexts by changing up your starting task and setups that prompts do.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Thoughts: ${answer}

---

Question: 20 birds migrate on a seasonal basis from one lake to another, searching for food. If they fly from lake Jim to lake Disney in one season, which is 50 miles apart, then the next season they fly from lake Disney to lake London, 60 miles apart, calculate the combined distance all of the birds have traveled in the two seasons.
Reasoning: Let's think step by step in order to calculate the combined distance traveled by the birds. First, we need to find the total distance traveled by the birds in the first season, which is from lake Jim to lake Disney. Then, we need to find the total distance traveled in the second season, from lake Disney to lake London. Finally, we can add these two distances together to find the combined distance t

"\n\n\nExperiment with contexts by changing up your starting task and setups that prompts do.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\nThoughts: ${answer}\n\n---\n\nQuestion: 20 birds migrate on a seasonal basis from one lake to another, searching for food. If they fly from lake Jim to lake Disney in one season, which is 50 miles apart, then the next season they fly from lake Disney to lake London, 60 miles apart, calculate the combined distance all of the birds have traveled in the two seasons.\nReasoning: Let's think step by step in order to\x1b calculate the combined distance traveled by the birds. First, we need to find the total distance traveled by the birds in the first season, which is from lake Jim to lake Disney. Then, we need to find the total distance traveled in the second season, from lake Disney to lake London. Finally, we can add these two distances together to find th